In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np 

review_data = pd.read_csv("/content/drive/My Drive/cmpe 255/yelp_review.csv")
busi_data = pd.read_csv("/content/drive/My Drive/cmpe 255/yelp_business.csv")

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (49) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
review_data = review_data[['business_id', 'text', 'stars']]
review_data.head()

,business_id,text,stars
0,buF9druCkbuXLX526sGELQ,Apparently Prides Osteria had a rough summer a...,4.0
1,RA4V8pr014UyUbDvI-LW2A,This store is pretty good. Not as great as Wal...,4.0
2,_sS2LBIGNT5NQb6PD1Vtjw,I called WVM on the recommendation of a couple...,5.0
3,0AzLzHfOJgL7ROwhdww2ew,I've stayed at many Marriott and Renaissance M...,2.0
4,8zehGz9jnxPqXtOc7KaJxA,The food is always great here. The service fro...,4.0


In [4]:
busi_data = busi_data[busi_data.categories.str.contains('restaurant|food', na=False)]
busi_data = busi_data[['business_id', 'name']]
busi_data.head()

,business_id,name
29,jGennaZUr2MsJyRhijNBfA,Legal Sea Foods
36,jx91IMdGOmLOo8h_F9z39g,Cleary's Restaurant & Spirits
41,NRPemqVb4qpWFF0Avq_6OQ,Eurasia Sushi Bar & Seafood
74,Mf9io8WkMyGautXuSSfGTg,Los Potrillos Mexican Restaurant & Bar
112,NoPXKdIwRzflOW-5exFAqA,1st Avenue Seafood by Pocha


In [5]:
review_data['business_id'] = review_data['business_id'].astype(str)
busi_data['business_id'] = busi_data['business_id'].astype(str)

data = review_data.merge(busi_data, on='business_id')
print(data.shape)
data.head()

(31447, 4)


,business_id,text,stars,name
0,EEHhKSxUvJkoPSzeGKkpVg,I work in the Pru and this is the most afforda...,5.0,Boston Chowda Co
1,EEHhKSxUvJkoPSzeGKkpVg,How do I love thee Chowda? Let me count the wa...,4.0,Boston Chowda Co
2,EEHhKSxUvJkoPSzeGKkpVg,"Oy. Food poisoning two times in a row, months ...",1.0,Boston Chowda Co
3,EEHhKSxUvJkoPSzeGKkpVg,Two Stars. Here is why - Quality Control. \n\...,2.0,Boston Chowda Co
4,EEHhKSxUvJkoPSzeGKkpVg,This place is a must! I recommend the lobster ...,4.0,Boston Chowda Co


In [6]:
import nltk 
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

data['text'] = data['text'].astype(str)
#print(data.head())
data['text'] = data['text'].str.lower()
#print(data.head())

# stop word removal
from nltk.corpus import stopwords
stop = stopwords.words('english')

data['text'] = data['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
data.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,business_id,text,stars,name
0,EEHhKSxUvJkoPSzeGKkpVg,work pru affordable tasty place food court. de...,5.0,Boston Chowda Co
1,EEHhKSxUvJkoPSzeGKkpVg,love thee chowda? let count ways love thy deli...,4.0,Boston Chowda Co
2,EEHhKSxUvJkoPSzeGKkpVg,"oy. food poisoning two times row, months apart...",1.0,Boston Chowda Co
3,EEHhKSxUvJkoPSzeGKkpVg,two stars. - quality control. lobster bisque u...,2.0,Boston Chowda Co
4,EEHhKSxUvJkoPSzeGKkpVg,place must! recommend lobster roll (stuffed lo...,4.0,Boston Chowda Co


In [7]:
import string
import nltk
nltk.download('wordnet')
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet, stopwords
from nltk import pos_tag, word_tokenize
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import WhitespaceTokenizer
import re

def pos_tagger(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:         
        return None
    
    
def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    # lemmatize text
    pos_tagged = nltk.pos_tag(nltk.word_tokenize(text))

    wordnet_tagged = list(map(lambda x: (x[0], pos_tagger(x[1])), pos_tagged))
    # print(wordnet_tagged)

    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            # if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:       
            # else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    text = " ".join(lemmatized_sentence)
    return(text)


def remove_html_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)


def clean_text(text):
    # remove html tags
    text = remove_html_tags(text)
    # lower text
    text = text.lower()
    # unicode characters
    text = text.encode('ascii', 'ignore').decode()
    # remove digits
    text = ''.join([i for i in text if not i.isdigit()])
    # remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # remove stop words
    stop_words = stopwords.words('english')
    text = ' '.join([word for word in text.split(' ') if word not in stop_words])
    return text

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [8]:
data['text'] = data.text.apply(clean_text)
data.head()

,business_id,text,stars,name
0,EEHhKSxUvJkoPSzeGKkpVg,work pru affordable tasty place food court dea...,5.0,Boston Chowda Co
1,EEHhKSxUvJkoPSzeGKkpVg,love thee chowda let count ways love thy delic...,4.0,Boston Chowda Co
2,EEHhKSxUvJkoPSzeGKkpVg,oy food poisoning two times row months apart c...,1.0,Boston Chowda Co
3,EEHhKSxUvJkoPSzeGKkpVg,two stars quality control lobster bisque used...,2.0,Boston Chowda Co
4,EEHhKSxUvJkoPSzeGKkpVg,place must recommend lobster roll stuffed lobs...,4.0,Boston Chowda Co


In [12]:
words = ["discomfort", "ache", "death", "allergy", "diarrhoea", "infection", "stomach", "loosemotion", "sick", "vomit", "nausea", "throw up", "threw up", "unhygienic", "pain", "flu", "stale", "poision", "motion", "upset", "indigestion", "acidity", "barf" ]

for w in words:
  w = lemmatize_text(w);

In [13]:
# creating a new feature for food poisoning based on words seen in review text
for index,row in data.iterrows() :
    if any(word in row['text'] for word in words) and row['stars'] < 3:
        data.loc[index,'FP'] = 1
    else:
        data.loc[index,'FP'] = 0


data.head()

,business_id,text,stars,name,FP
0,EEHhKSxUvJkoPSzeGKkpVg,work pru affordable tasty place food court dea...,5.0,Boston Chowda Co,0.0
1,EEHhKSxUvJkoPSzeGKkpVg,love thee chowda let count ways love thy delic...,4.0,Boston Chowda Co,0.0
2,EEHhKSxUvJkoPSzeGKkpVg,oy food poisoning two times row months apart c...,1.0,Boston Chowda Co,0.0
3,EEHhKSxUvJkoPSzeGKkpVg,two stars quality control lobster bisque used...,2.0,Boston Chowda Co,0.0
4,EEHhKSxUvJkoPSzeGKkpVg,place must recommend lobster roll stuffed lobs...,4.0,Boston Chowda Co,0.0


In [14]:
from sklearn.model_selection import train_test_split

# split the data into train and test set
train, test = train_test_split(data, test_size=0.2, random_state=42, shuffle=True)

In [15]:
train_X = train['text']   # '0' corresponds to Texts/Reviews
train_y = train['FP']   # '1' corresponds to Label (1 - positive and 0 - negative)
test_X = test['text'] 
test_y = test['FP']

In [16]:
from time import time
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
t = time()  # not compulsory

# loading CountVectorizer
tf_vectorizer = CountVectorizer() # or term frequency

X_train_tf = tf_vectorizer.fit_transform(train_X)
duration = time() - t
print("Time taken to extract features from training data : %f seconds" % (duration))
print("n_samples: %d, n_features: %d" % X_train_tf.shape)

Time taken to extract features from training data : 1.335057 seconds
n_samples: 25157, n_features: 42138


In [17]:
t = time()
X_test_tf = tf_vectorizer.transform(test_X)

duration = time() - t
print("Time taken to extract features from test data : %f seconds" % (duration))
print("n_samples: %d, n_features: %d" % X_test_tf.shape)

Time taken to extract features from test data : 0.315587 seconds
n_samples: 6290, n_features: 42138


In [18]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
# build naive bayes classification model
t = time()

naive_bayes_classifier = MultinomialNB()
naive_bayes_classifier.fit(X_train_tf, train_y)

training_time = time() - t
print("train time: %0.3fs" % training_time)

train time: 0.028s


In [19]:
# predict the new document from the testing dataset
t = time()
y_pred = naive_bayes_classifier.predict(X_test_tf)

test_time = time() - t
print("Naive Bayes classifier")
print("test time:  %0.3fs" % test_time)

# compute the performance measures
score_nb = metrics.accuracy_score(test_y, y_pred)
print("accuracy:   %0.3f" % score_nb)

print(metrics.classification_report(test_y, y_pred,
                                            target_names=['Positive', 'Negative']))

print("confusion matrix:")
print(metrics.confusion_matrix(test_y, y_pred))

print('------------------------------')

Naive Bayes classifier
test time:  0.009s
accuracy:   0.981
              precision    recall  f1-score   support

    Positive       0.98      1.00      0.99      6172
    Negative       0.33      0.03      0.05       118

    accuracy                           0.98      6290
   macro avg       0.66      0.51      0.52      6290
weighted avg       0.97      0.98      0.97      6290

confusion matrix:
[[6166    6]
 [ 115    3]]
------------------------------


In [20]:
# building knn classifier
from sklearn.neighbors import KNeighborsClassifier
t = time()

classifier = KNeighborsClassifier(n_neighbors=150, algorithm='auto')
classifier.fit(X_train_tf, train_y)

training_time = time() - t
print("train time: %0.3fs" % training_time)

train time: 0.012s


In [21]:
t = time()
predicted_labels = classifier.predict(X_test_tf)


In [22]:
# compute the performance measures
print("KNN classifier")
test_time = time() - t
print("test time:  %0.3fs" % test_time)
score_knn = metrics.accuracy_score(test_y, predicted_labels)
print("accuracy:   %0.3f" % score_knn)

print(metrics.classification_report(test_y, predicted_labels,
                                            target_names=['Positive', 'Negative']))

print("confusion matrix:")
print(metrics.confusion_matrix(test_y, predicted_labels))

print('------------------------------')

KNN classifier
test time:  10.629s
accuracy:   0.981
              precision    recall  f1-score   support

    Positive       0.98      1.00      0.99      6172
    Negative       0.00      0.00      0.00       118

    accuracy                           0.98      6290
   macro avg       0.49      0.50      0.50      6290
weighted avg       0.96      0.98      0.97      6290

confusion matrix:
[[6172    0]
 [ 118    0]]
------------------------------


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
